<h1>Importing Libraries</h1>

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.model_selection import train_test_split

<h1>Loading Data</h1>

In [3]:
# Load data
train = pd.read_parquet('../train.parquet')
test = pd.read_parquet('../test.parquet')
sample_sub = pd.read_csv('../sample_submission.csv')

<h1>Precomputations</h1>

In [4]:
# Pre-calculate global means from training data
iv_columns = [col for col in test.columns if col.startswith(('call_iv_', 'put_iv_'))]
global_means = {}
for col in iv_columns:
    if col in train.columns:
        global_means[col] = train[col].mean()
overall_mean = np.mean(list(global_means.values())) if global_means else 0.2

# Create strike dictionary
strike_dict = {}
for col in iv_columns:
    strike = col.split('_')[-1]
    if strike not in strike_dict:
        strike_dict[strike] = {'call': None, 'put': None}
    if col.startswith('call_iv_'):
        strike_dict[strike]['call'] = col
    else:
        strike_dict[strike]['put'] = col

<h1>Black Scholes function</h1>

In [5]:
# Black-Scholes European option price calculator
def black_scholes_price(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:  # put
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return price

<h1>Implied volatility calculation using Newton-Raphson</h1>

In [6]:
# Implied volatility calculation using Newton-Raphson
def calculate_iv(row, strike, option_type, risk_free_rate=0.05, T=1/52):
    S = row['underlying']
    K = float(strike)
    
    # Skip if underlying price is missing
    if pd.isna(S) or S <= 0:
        return np.nan
        
    # Get market price from features (anonymized features may contain price info)
    # Use X0 as proxy for option price if available
    price_col = 'X0' if 'X0' in row else None
    if price_col and not pd.isna(row[price_col]):
        market_price = row[price_col]
    else:
        return np.nan
        
    # Newton-Raphson implementation
    sigma = 0.2  # Initial guess
    max_iter = 100
    tolerance = 1e-8
    
    for i in range(max_iter):
        price = black_scholes_price(S, K, T, risk_free_rate, sigma, option_type)
        vega = S * norm.pdf((np.log(S/K) + (risk_free_rate + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))) * np.sqrt(T)
        
        if vega < 1e-10:
            break
            
        diff = market_price - price
        if abs(diff) < tolerance:
            break
            
        sigma += diff / vega
        
    return sigma

<h1>volatility smile fitting</h1>

In [7]:
# Enhanced volatility smile fitting
def fit_volatility_smile(row, strike_dict, S):
    moneyness = []
    ivs = []
    
    for strike, cols in strike_dict.items():
        call_col = cols['call']
        put_col = cols['put']
        
        if call_col in row and put_col in row:
            # Prefer non-nan IV value
            if not pd.isna(row[call_col]):
                iv_val = row[call_col]
            elif not pd.isna(row[put_col]):
                iv_val = row[put_col]
            else:
                continue
                
            strike_val = float(strike)
            if S > 0:  # Avoid division by zero
                m = strike_val / S
                moneyness.append(m)
                ivs.append(iv_val)
    
    if len(moneyness) < 3:
        return None, None
    
    # Fit quadratic polynomial
    try:
        coeff = np.polyfit(moneyness, ivs, 2)
        return np.poly1d(coeff), np.mean(ivs)
    except:
        return None, np.mean(ivs)

<h1>Prediction Function</h1>

In [8]:
# Main prediction function
def predict_iv(data, train_data=train):
    data = data.copy()
    
    # Phase 1: Put-call parity
    for strike, cols in strike_dict.items():
        call_col = cols['call']
        put_col = cols['put']
        
        if call_col in data.columns and put_col in data.columns:
            # Fill calls using puts
            call_mask = data[call_col].isna() & data[put_col].notna()
            data.loc[call_mask, call_col] = data.loc[call_mask, put_col]
            
            # Fill puts using calls
            put_mask = data[put_col].isna() & data[call_col].notna()
            data.loc[put_mask, put_col] = data.loc[put_mask, call_col]
    
    # Phase 2: Black-Scholes for remaining missing values
    for idx, row in data.iterrows():
        S = row['underlying']
        if pd.isna(S) or S <= 0:
            continue
            
        # Fit volatility smile for current row
        poly, mean_iv = fit_volatility_smile(row, strike_dict, S)
        
        for strike, cols in strike_dict.items():
            call_col = cols['call']
            put_col = cols['put']
            
            # Only process if both columns exist
            if call_col not in data.columns or put_col not in data.columns:
                continue
                
            # Check if both are still missing
            if pd.isna(data.at[idx, call_col]) and pd.isna(data.at[idx, put_col]):
                strike_val = float(strike)
                
                # Try to use volatility smile fit
                if poly is not None and S > 0:
                    m = strike_val / S
                    pred_iv = poly(m)
                elif not pd.isna(mean_iv):
                    pred_iv = mean_iv
                else:
                    # Calculate IV using Black-Scholes as fallback
                    try:
                        pred_iv = calculate_iv(row, strike, 'call')
                    except:
                        pred_iv = global_means.get(call_col, overall_mean)
                
                # Assign predicted IV to both call and put
                if not pd.isna(pred_iv):
                    data.at[idx, call_col] = pred_iv
                    data.at[idx, put_col] = pred_iv
    
    # Phase 3: Global imputation for any remaining missing values
    for col in iv_columns:
        if col in data.columns:
            fill_value = global_means.get(col, overall_mean)
            data[col] = data[col].fillna(fill_value)
    
    return data

<h1>Validation Score</h1>

In [9]:
# Create validation split
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

# Apply to validation set
val_pred = predict_iv(val_df)

# Calculate MSE
mse_vals = []
for col in iv_columns:
    if col in val_df.columns and col in val_pred.columns:
        # Only calculate where we have ground truth
        valid_mask = val_df[col].notna()
        if valid_mask.any():
            se = (val_df.loc[valid_mask, col] - val_pred.loc[valid_mask, col]) ** 2
            mse_vals.append(se.mean())

validation_mse = np.mean(mse_vals) if mse_vals else float('nan')
print(f"Validation MSE: {validation_mse:.10f}")

Validation MSE: 0.0000000000


<h1>Test data predictions</h1>

In [10]:
# Apply to test set
test_pred = predict_iv(test)


<h1>Submissions</h1>

In [11]:
# Prepare submission
submission = test_pred[['timestamp'] + iv_columns].copy()
submission.columns = sample_sub.columns
submission.to_csv('submission.csv', index=False)

# Verify results
print("\nFinal Submission Preview:")
print(submission.head())
print(f"\nMissing values: {submission.isna().sum().sum()}")
print(f"Submission shape: {submission.shape}")
print(f"Validation MSE: {validation_mse:.10f}")


Final Submission Preview:
   timestamp  call_iv_24000  call_iv_24100  call_iv_24200  call_iv_24300  \
0          0       0.280939       0.268189       0.256631       0.247051   
1          1       0.270276       0.263275       0.258893       0.246509   
2          2       0.271578       0.251731       0.237865       0.224018   
3          3       0.241888       0.232431       0.220505       0.210733   
4          4       0.235328       0.229432       0.222983       0.214126   

   call_iv_24400  call_iv_24500  call_iv_24600  call_iv_24700  call_iv_24800  \
0       0.242149       0.233827       0.232439       0.228517       0.222997   
1       0.244875       0.236401       0.233548       0.228209       0.233722   
2       0.214869       0.204580       0.194604       0.188052       0.184880   
3       0.198602       0.186190       0.176759       0.166394       0.161561   
4       0.203883       0.199485       0.192603       0.187150       0.183532   

   ...  put_iv_24600  put_iv_24700 